In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, ElasticNet
import csv
import seaborn as sns

C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
path1 = f"./data/round_2/prices_round_2_day_-1.csv"
path2 = f"./data/round_2/prices_round_2_day_0.csv"
path3 = f"./data/round_2/prices_round_2_day_1.csv"
results_path = f"./data/round_2/algo_results.csv"

In [3]:
data1 = pd.read_csv(path1, delimiter=";", parse_dates=True)
data2 = pd.read_csv(path2, delimiter=";", parse_dates=True)
data3 = pd.read_csv(path3, delimiter=";", parse_dates=True)
results_data = pd.read_csv(results_path, delimiter=";", parse_dates=True)

In [4]:
data = pd.concat([data1, data2, data3], ignore_index=True)
data

,timestamp,ORCHIDS,TRANSPORT_FEES,EXPORT_TARIFF,IMPORT_TARIFF,SUNLIGHT,HUMIDITY,DAY
0,0,1200.00,1.5,10.5,-2.0,2500.0000,79.00000,-1
1,100,1201.75,1.5,9.5,-2.0,2499.4197,79.00410,-1
2,200,1201.75,1.5,9.5,-2.0,2498.8457,79.00821,-1
3,300,1201.75,1.5,9.5,-2.0,2498.2780,79.01234,-1
4,400,1201.75,1.5,9.5,-2.0,2497.7166,79.01649,-1
...,...,...,...,...,...,...,...,...
29998,999600,1036.25,0.9,9.5,-5.0,2299.9436,69.93549,1
29999,999700,1036.25,0.9,9.5,-5.0,2299.9556,69.95153,1
30000,999800,1036.25,0.9,9.5,-5.0,2299.9690,69.96762,1
30001,999900,1034.25,0.9,9.5,-5.0,2299.9840,69.98378,1


In [5]:
data_sun = data[['SUNLIGHT']]
data_sun

,SUNLIGHT
0,2500.0000
1,2499.4197
2,2498.8457
3,2498.2780
4,2497.7166
...,...
29998,2299.9436
29999,2299.9556
30000,2299.9690
30001,2299.9840


In [6]:
data_hum = data[['HUMIDITY']]
data_hum

,HUMIDITY
0,79.00000
1,79.00410
2,79.00821
3,79.01234
4,79.01649
...,...
29998,69.93549
29999,69.95153
30000,69.96762
30001,69.98378


In [7]:
def add_lags(parsed_data, col):
    lags = 3
    for i in range(1, lags + 1):
        parsed_data[f'lag{i}'] = parsed_data[col].shift(i)

    parsed_data.dropna(inplace=True)
    return parsed_data

In [8]:
lag_sun_data = add_lags(data_sun, 'SUNLIGHT')
lag_sun_data

C:\Users\HP\AppData\Local\Temp\ipykernel_5992\411776612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed_data[f'lag{i}'] = parsed_data[col].shift(i)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,SUNLIGHT,lag1,lag2,lag3
3,2498.2780,2498.8457,2499.4197,2500.0000
4,2497.7166,2498.2780,2498.8457,2499.4197
5,2497.1614,2497.7166,2498.2780,2498.8457
6,2496.6125,2497.1614,2497.7166,2498.2780
7,2496.0698,2496.6125,2497.1614,2497.7166
...,...,...,...,...
29998,2299.9436,2299.9330,2299.9238,2299.9160
29999,2299.9556,2299.9436,2299.9330,2299.9238
30000,2299.9690,2299.9556,2299.9436,2299.9330
30001,2299.9840,2299.9690,2299.9556,2299.9436


In [9]:
lag_hum_data = add_lags(data_hum, 'HUMIDITY')
lag_hum_data

C:\Users\HP\AppData\Local\Temp\ipykernel_5992\411776612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed_data[f'lag{i}'] = parsed_data[col].shift(i)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,HUMIDITY,lag1,lag2,lag3
3,79.01234,79.00821,79.00410,79.00000
4,79.01649,79.01234,79.00821,79.00410
5,79.02066,79.01649,79.01234,79.00821
6,79.02485,79.02066,79.01649,79.01234
7,79.02906,79.02485,79.02066,79.01649
...,...,...,...,...
29998,69.93549,69.91952,69.90361,69.88776
29999,69.95153,69.93549,69.91952,69.90361
30000,69.96762,69.95153,69.93549,69.91952
30001,69.98378,69.96762,69.95153,69.93549


In [10]:
x_hum = lag_hum_data[['lag1', 'lag2', 'lag3']]
y_hum = lag_hum_data['HUMIDITY']

print(x_hum)
print()
print(y_hum)

           lag1      lag2      lag3
3      79.00821  79.00410  79.00000
4      79.01234  79.00821  79.00410
5      79.01649  79.01234  79.00821
6      79.02066  79.01649  79.01234
7      79.02485  79.02066  79.01649
...         ...       ...       ...
29998  69.91952  69.90361  69.88776
29999  69.93549  69.91952  69.90361
30000  69.95153  69.93549  69.91952
30001  69.96762  69.95153  69.93549
30002  69.98378  69.96762  69.95153

[30000 rows x 3 columns]

3        79.01234
4        79.01649
5        79.02066
6        79.02485
7        79.02906
           ...   
29998    69.93549
29999    69.95153
30000    69.96762
30001    69.98378
30002    70.00000
Name: HUMIDITY, Length: 30000, dtype: float64


In [11]:
x_sun = lag_sun_data[['lag1', 'lag2', 'lag3']]
y_sun = lag_sun_data['SUNLIGHT']

print(x_sun)
print()
print(y_sun)

            lag1       lag2       lag3
3      2498.8457  2499.4197  2500.0000
4      2498.2780  2498.8457  2499.4197
5      2497.7166  2498.2780  2498.8457
6      2497.1614  2497.7166  2498.2780
7      2496.6125  2497.1614  2497.7166
...          ...        ...        ...
29998  2299.9330  2299.9238  2299.9160
29999  2299.9436  2299.9330  2299.9238
30000  2299.9556  2299.9436  2299.9330
30001  2299.9690  2299.9556  2299.9436
30002  2299.9840  2299.9690  2299.9556

[30000 rows x 3 columns]

3        2498.2780
4        2497.7166
5        2497.1614
6        2496.6125
7        2496.0698
           ...    
29998    2299.9436
29999    2299.9556
30000    2299.9690
30001    2299.9840
30002    2300.0000
Name: SUNLIGHT, Length: 30000, dtype: float64


In [12]:
model_hum = LinearRegression()
model_hum.fit(x_hum, y_hum)
pred = model_hum.predict(x_hum)
lag_hum_data['pred'] = pred
lag_hum_data

C:\Users\HP\AppData\Local\Temp\ipykernel_5992\383282565.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lag_hum_data['pred'] = pred


,HUMIDITY,lag1,lag2,lag3,pred
3,79.01234,79.00821,79.00410,79.00000,79.008022
4,79.01649,79.01234,79.00821,79.00410,79.012152
5,79.02066,79.01649,79.01234,79.00821,79.016302
6,79.02485,79.02066,79.01649,79.01234,79.020473
7,79.02906,79.02485,79.02066,79.01649,79.024663
...,...,...,...,...,...
29998,69.93549,69.91952,69.90361,69.88776,69.919792
29999,69.95153,69.93549,69.91952,69.90361,69.935763
30000,69.96762,69.95153,69.93549,69.91952,69.951804
30001,69.98378,69.96762,69.95153,69.93549,69.967896


In [13]:
print("Coefficients:", model_hum.coef_)
print("Intercept:", model_hum.intercept_)
print("Accuracy:", model_hum.score(x_hum, y_hum))

Coefficients: [ 1.01254311e+00 -3.38481111e-06 -1.25578231e-02]
Intercept: 0.0011386486631153048
Accuracy: 0.9999349822110898


In [14]:
model_sun = LinearRegression()
model_sun.fit(x_sun, y_sun)
pred = model_sun.predict(x_sun)
lag_sun_data['pred'] = pred
lag_sun_data

C:\Users\HP\AppData\Local\Temp\ipykernel_5992\2039741743.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lag_sun_data['pred'] = pred


,SUNLIGHT,lag1,lag2,lag3,pred
3,2498.2780,2498.8457,2499.4197,2500.0000,2498.806805
4,2497.7166,2498.2780,2498.8457,2499.4197,2498.239492
5,2497.1614,2497.7166,2498.2780,2498.8457,2497.678480
6,2496.6125,2497.1614,2497.7166,2498.2780,2497.123665
7,2496.0698,2496.6125,2497.1614,2497.7166,2496.575149
...,...,...,...,...,...
29998,2299.9436,2299.9330,2299.9238,2299.9160,2299.931403
29999,2299.9556,2299.9436,2299.9330,2299.9238,2299.942088
30000,2299.9690,2299.9556,2299.9436,2299.9330,2299.954173
30001,2299.9840,2299.9690,2299.9556,2299.9436,2299.967658


In [16]:
print("Coefficients:", model_sun.coef_)
print("Intercept:", model_sun.intercept_)
print("Accuracy:", model_sun.score(x_hum, y_hum))

Coefficients: [ 1.03037314e+00 -2.25013886e-06 -3.03795021e-02]
Intercept: 0.017696349974357872
Accuracy: 0.9999315297997203
